In [ ]:
import pandas as pd
import zipfile
import seaborn as sns

# Join Review

In this notebook we'll briefly review [joining tables as discussed in Data 8](https://www.inferentialthinking.com/chapters/08/4/Joining_Tables_by_Columns.html).

Often data is spread across two tables. Joining provides us with a way to naturally combine related tables.

Let's start by reading data from the given zip file. To showcase how to do read data that is inside zip files, we're going to use the `zipfile` module. Doing so will allow us to avoid needing to unzip the data. Running the cell below, we see that the zip file contains `elections.csv` and `presidents.csv`.

In [ ]:
join_demo_filename = "lec5_join_demo_data.zip"
my_zip = zipfile.ZipFile(join_demo_filename, 'r')
list_names = [f.filename for f in my_zip.filelist]
list_names

We could call `my_zip.extractall()` to unzip the files, but we won't. Instead, we'll read directly from the zip file itself.

In [ ]:
with my_zip.open("elections.csv") as f:
    elections = pd.read_csv(f)
    
elections.head(5)

In [ ]:
with my_zip.open("presidents.csv") as f:
    presidents = pd.read_csv(f)
    
presidents.head(5)

To join tables `df` and `df2`, we call the function `df.merge(df2)`. Merge is just the word that the authors of pandas picked for joining tables. I don't know why.

Note: Unfortunately, Pandas also has a function called `df.join`. This is a limited version of `merge`. For the sake of generality, we will only use `merge` in this class.

I can use the merge function to combine these two tables:

In [ ]:
elections.merge(presidents, 
            how = "inner",
            left_on = "Candidate", right_on = "President")

Notice that:
1. The output dataframe only contains rows that have names in both tables.  For example, presidents before 1824 do not appear in the joined table because there was no popular vote before 1824.
1. The name `Andrew Jackson` occurred three times in the election table and shows up three times in the output. 
1. `Grover Cleveland` occurs six times! Twice for every election he was in. This is because he appears three times in the `elections` table and twice in the `presidents` table. This results in 3 x 2 = 6 combinations.
1. Several presidents are missing because their names are not an exact match. For example, John F. Kennedy is "John Kennedy" in the elections table and "John F. Kennedy" in the presidents table.

With the caveats above in mind, this merged DataFrame is handy because we can use it to plot, e.g. the age of each president when they were elected.

In [ ]:
joined = elections.merge(presidents, 
            how = "inner",
            left_on = "Candidate", right_on = "President")

winners = joined.query("Result == 'win'").copy()
winners["Birthyear"] = winners["Date of birth"].str.split(',').str[1].map(int)
winners["Age"] = winners["Year"] - winners["Birthyear"]
sns.lmplot(data=winners, x="Year", y="Age");

### How could we fix the duplicate Grover Cleveland?

We could group by name/candidate and take only the first:

In [ ]:
(
        elections.merge(presidents, 
            how = "inner",
            left_on = "Candidate", right_on = "President")
        .groupby(['Candidate', 'Year']).first().reset_index()
)

### Right Join

The above join was an inner join.  What if we wanted to keep all of the presidents and leave missing data for years when there was no popular vote? In this case we'd do a "right" join, where we make sure to include EVERY row from our right dataframe, in this case `presidents`.

In [ ]:
elections.merge(presidents, 
            how = "right",
            left_on = "Candidate", right_on = "President")

Similarly, if we also want to include candidates not in the presidents table (e.g. because they had never won), we can use a "left" join.

In [ ]:
elections.merge(presidents, 
            how = "left",
            left_on = "Candidate", right_on = "President")

If we wanted to keep both, we can instead do an "outer join".

In [ ]:
elections.merge(presidents, 
            how = "outer",
            left_on = "Candidate", right_on = "President")